In [26]:
import pickle
from math import floor

In [8]:
args = {
    "true_events": "/home/ibis/gregor.sturm/nanopore/own/notebooks/03_pipeline/david_alignment_true_events.pickle",
    "ref": "/home/ibis/gregor.sturm/nanopore/david_eccles_bc_ideas/mouse_ref.fa",
}

# args = {
#     "true_events": "/home/ibis/gregor.sturm/nanopore/own/notebooks/03_pipeline/lambda_alignment_true_events.pickle",
#     "ref": "/home/ibis/gregor.sturm/nanopore/own/notebooks/03_pipeline/lambda_ref.fasta",
# }

In [9]:
NMER = 3

In [10]:
!pwd

/home/ibis/gregor.sturm/nanopore/own/notebooks/04_error_correction


In [11]:
true_events = pickle.load(open(args["true_events"], 'rb'))

In [12]:
ref_file = args["ref"]
test = !cat {ref_file} | grep ">"
print(test)
ref = !cat {ref_file} | grep -v ">"
ref = ref[0]
print(ref[:100])

['>mmusMT_PCR1']
GTTAATGTAGCTTAATAACAAAGCAAAGCACTGAAAATGCTTAGATGGATAATTGTATCCCATAAACACAAAGGTTTGGTCCTGGCCTTATAATTAATTA


In [46]:
def get_context(event, start, end):
    """get the nt-context around the kmer. 
    E.g. start=-20: get context 20nt before the reference point
    (mid of kmer)"""
    ref_pos = event["ref_position"]
    start = ref_pos + start
    end = ref_pos + end
    return ref[start:end]

def get_context_before(event, n):
    """get n nucleotides before the start of the kmer"""
    length = len(event["kmer"])
    offset = -floor(length/2)
    return get_context(event, -n+offset, offset)
    
def get_context_after(event, n):
    length = len(event["kmer"])
    offset = floor(length/2) + 1
    return get_context(event, offset, offset+n)

In [50]:
test_event = true_events[0]
assert test_event['kmer'] == get_context(test_event, -2, 2+1), "invalid reference"
assert (get_context_before(test_event, 20) 
        + test_event['kmer'] 
        + get_context_after(test_event, 20)) == (
    get_context(test_event, -22, 22+1))

In [51]:
# print(" " * 20 + test_event["kmer"])
# print(get_context_before(test_event, 20))
# print(get_context(test_event, -22, 22+1))
# print(" " * 25 + get_context_after(test_event, 20))

                    TAAAT
GTTATACCTCTTAGGGTTGG
GTTATACCTCTTAGGGTTGGTAAATTTCGTGCCAGCCACCGCGGT
                         TTCGTGCCAGCCACCGCGGT


'TATACCTCTTAGGGTTGGTAAATTTCGTGCCAGCCACCGC'